In [ ]:
import pandas as pd
import numpy as np

# 3 parts:
# kostenschätzungs
# ausgabenberechnung (bilanz berechnen => schätzung vs. tatsächliche kosten)
# kostenvergleich

In [ ]:
from building_cost_analysis.settings import (
    DIR_ANGEBOTE,
    PATH_GEWERKE,
    PATH_AUSGABEN,
    PATH_KOSTENUEBERSICHT
)
from building_cost_analysis.parser import (
    get_df_gewerke,
    get_offer_file_paths,
    get_offer_file_paths_and_sheets,
    get_df_offer,
    get_df_offers_combined,
    get_list_df_offers
)

from building_cost_analysis.calc import (
    append_rolled_up_sum_to_df,
    get_df_duplicated_positions,
    compare_offers,
    get_missing_positions
)

# get_df_offers_combined
list_paths_sheets = get_offer_file_paths_and_sheets(DIR_ANGEBOTE)
df_gewerke = get_df_gewerke(PATH_GEWERKE)

In [ ]:
list_paths_sheets

In [ ]:
list_paths_sheets[-4]

In [ ]:
def get_paths_sheets_by_name(list_path_sheets: list, gewerk: str, sheet: str):
    for path_sheet in list_path_sheets:
        path_offer = path_sheet['path_offer'].replace('.xlsx', '')
        if path_offer.endswith(gewerk) and sheet == path_sheet['sheet_name']:
            return path_sheet



In [ ]:
# pick gewerke
list_combine_path_sheets = []
baumeister = get_paths_sheets_by_name(list_paths_sheets, "Baumeister", "Mueller_Minimum")
bautischler = get_paths_sheets_by_name(list_paths_sheets, "Bautischler", "Julia_Minimum")
bodenleger = get_paths_sheets_by_name(list_paths_sheets, "Bodenleger", "Julia_Minimum")
dachdecker = get_paths_sheets_by_name(list_paths_sheets, "Dachdecker", "Wurzrainer_Minimum_2")
fenster = get_paths_sheets_by_name(list_paths_sheets, "Fenster", "Kitzfenster_Minimum")
fliesenleger = get_paths_sheets_by_name(list_paths_sheets, "Fliesenleger", "Kostenschaetzung_Minimum")
installateur = get_paths_sheets_by_name(list_paths_sheets, "Installateur", "Kracher_Minimum")
klaer = get_paths_sheets_by_name(list_paths_sheets, "Klaeranlage", "Hechenbichler_1")
kostensch = get_paths_sheets_by_name(list_paths_sheets, "Kostenschaetzung", "Kostenschaetzung_Minimum_2")
nebenkosten = get_paths_sheets_by_name(list_paths_sheets, "Nebenkosten", "Schaetzung_Minimum")
zimmerer = get_paths_sheets_by_name(list_paths_sheets, "Zimmerer", "Foidl_Minimum")

list_combine_path_sheets.append(baumeister)
list_combine_path_sheets.append(bautischler)
list_combine_path_sheets.append(bodenleger)
list_combine_path_sheets.append(dachdecker)
list_combine_path_sheets.append(fenster)
list_combine_path_sheets.append(fliesenleger)
list_combine_path_sheets.append(installateur)
list_combine_path_sheets.append(klaer)
list_combine_path_sheets.append(kostensch)
list_combine_path_sheets.append(nebenkosten)
list_combine_path_sheets.append(zimmerer)

In [ ]:

list_combine_path_sheets

In [ ]:
df_offers = get_df_offers_combined(list_combine_path_sheets)
df_combined = append_rolled_up_sum_to_df(df_offers, df_gewerke)

df_combined.loc[df_combined['level'] == 0][['summed_up']].sum() * 1.1# 5 % sicherheitsreserve

In [ ]:
display_cols=['level', 'id_0', 'id_1', 'Beschreibung', 'Brutto', 'E-Preis', 'Einheit', 'Gesamtpreis', 'Menge', 'summed_up']

df_combined.loc[df_combined['level'] == 1][display_cols]

In [ ]:
df_combined.to_csv('./data/test.csv')

In [ ]:
df_combined[display_cols].to_csv('./data/current.csv')

In [ ]:
df_combined.columns

In [ ]:
df_combined.columns

def add_bill_amounts_to_df(df_estimates: pd.DataFrame, path_ausgaben: str) -> None:
    df_estimates['paid'] = None
    df_estimates['num_bills'] = None
    df_estimates['paid_summed_up'] = None
    df_bills = pd.read_excel(path_ausgaben, skiprows=7)
    df_bills = df_bills.rename(columns={"Kosten [€]\nbezahlt": "amount"})

    for id, row in df_combined.iterrows():
        df_bills_id = df_bills.loc[df_bills['ID'] == id]
        if df_bills_id.empty:
            continue
        df_estimates.loc[df_estimates.index == id, 'paid'] = df_bills_id['amount'].sum()
        df_estimates.loc[df_estimates.index == id, 'num_bills'] = df_bills_id['amount'].count()
add_bill_amounts_to_df(df_combined, PATH_AUSGABEN)

In [ ]:
df_combined.to_csv('./data/test.csv')

In [ ]:
    #PATH_AUSGABEN,
    #PATH_KOSTENUEBERSICHT


import openpyxl
from openpyxl.styles import Font, Color

COL_ID = "B"
COL_ID0_PRICE = "F"
COL_ID3_PRICE = "G"
COL_ID4_PRICE = "H"

COL_ID0_PAID = "I"
COL_ID3_PAID = "J"
COL_ID4_PAID = "K"

COL_ID0_FONT = "C" 
COL_ID3_FONT = "D"
COL_ID4_FONT = "E"

# Define variable to load the dataframe
def insert_estimates_into_cost_overview(path_kostenuebersicht: str, df_estimate: pd.DataFrame):
        workbook = openpyxl.load_workbook(filename=path_kostenuebersicht)
        sheet = workbook.active
        #print(sheet['C4'].font)
        for row in range(0, sheet.max_row):
                id_cell = COL_ID + str(row + 1)
                current_id = sheet[id_cell].value
                df_row_estimate = df_estimate.loc[df_estimate.index == current_id]
                if df_row_estimate.empty:
                    continue
                price_existing = df_row_estimate.iloc[0]['prices_existing']
                id_level = df_row_estimate.iloc[0]['level'] 

                # apply estimate + paid
                if id_level < 3:
                    id_cell_estimate = COL_ID0_PRICE + str(row + 1)
                    id_cell_paid = COL_ID0_PAID + str(row + 1)
                elif id_level == 3:
                    id_cell_estimate = COL_ID3_PRICE + str(row + 1)
                    id_cell_paid = COL_ID3_PAID + str(row + 1)
                elif id_level == 4:
                    id_cell_estimate = COL_ID4_PRICE + str(row + 1)
                    id_cell_paid = COL_ID4_PAID + str(row + 1)

                if price_existing:
                    sheet[id_cell_estimate] = (int(df_row_estimate.iloc[0]['summed_up']))
                
                if df_row_estimate.iloc[0]['paid'] is not None:
                    sheet[id_cell_paid] = (int(df_row_estimate.iloc[0]['paid']))


                #apply style
                if id_level < 3:
                    id_font = COL_ID0_FONT + str(row + 1)
                elif id_level == 3:
                    id_font = COL_ID3_FONT + str(row + 1)
                elif id_level == 4:
                    id_font = COL_ID4_FONT + str(row + 1)
                cell_font = sheet[id_font].font

                sheet[id_cell_estimate].font = Font(sz=cell_font.sz, b=cell_font.b, i=cell_font.i, underline=cell_font.underline)
            
        workbook.save(filename=PATH_KOSTENUEBERSICHT)

insert_estimates_into_cost_overview(PATH_KOSTENUEBERSICHT, df_combined)